In [2]:
import os
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings


os.environ["AZURE_OPENAI_API_KEY"] = "3293d148178a4cf39f5b3026cb621eb1" 
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://fish-health-bsc-resource-aiservices1116524949.openai.azure.com/"

model = AzureChatOpenAI(
     openai_api_version="2023-07-01-preview",
     azure_deployment="gpt-4",
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2023-07-01-preview",
)

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

TAVILY - Built in search engine tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

search.invoke("Hvordan er været i Bergen, Norge")

[{'url': 'https://www.timeanddate.no/vaer/norge/bergen',
  'content': 'Været i Bergen, Norge. Sol og måne. Været i dag Time for time 14-dagers varsel I går/været som var Klima (gjennomsnitt) Nå.'},
 {'url': 'https://www.yr.no/nb/værvarsel/daglig-tabell/1-92416/Norge/Vestland/Bergen/Bergen',
  'content': 'Værvarsel for Bergen de neste 10 dagene. Gå til innholdet. En tjeneste fra Meteorologisk institutt og NRK ... Det ser ut til at JavaScript er skrudd av i nettleseren din. For å få tilgang til alt av innhold på Yr anbefaler vi at du skrur på JavaScript. Været nå ...'},
 {'url': 'https://www.yr.no/nb/værvarsel/daglig-tabell/1-513376/Norge/Vestland/Bergen',
  'content': 'Værvarsel for Bergen kommune de neste 10 dagene. Gå til innholdet. En tjeneste fra Meteorologisk institutt og NRK ... Været nå . Temperatur 7 ° 7 ° ... Yr er en tjeneste fra. Redaktør: Ingrid Støver JensenAnsvarlig redaktør: Vibeke Fürst HaugenMeteorologisk ansvarlig: Roar Skålin.'},
 {'url': 'https://www.timeanddate.no/v

Adding a vectorestore with the Langsmith documentation

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = Chroma.from_documents(documents, AzureOpenAIEmbeddings())
retriever = vector.as_retriever()

In [7]:
retriever.get_relevant_documents("how to upload a dataset")[0]

Document(page_content="about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn how to create an account, obtain an API key, and configure your environment.Pricing: Learn about the pricing model for LangSmith.Self-Hosting: Learn about self-hosting options for LangSmith.Proxy: Learn about the proxy capabilities of LangSmith.Tracing: Learn about the tracing capabilities of LangSmith.Evaluation: Learn about the evaluation capabilities of LangSmith.Prompt Hub Learn about the Prompt Hub, a prompt management tool built into LangSmith.Additional Resources\u200bLangSmith Cookbook: A collection of tutorials and end-to-end walkthroughs using LangSmith.LangChain Python: Docs for the Python LangChain library.LangChain Python API Reference: documentation to review the core APIs of LangChain.LangChain JS: Docs for the TypeScript LangChain libraryDiscord: Join us on our Discord to discuss all things LangChain!Contact SalesIf 

In [8]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [12]:
tools = [search, retriever_tool]

In [14]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(model, tools, prompt)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [18]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at e

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith, developed by LangChain, is a platform specifically designed to help with building production-grade LLM applications. One of its features includes providing support for testing. This platform allows you to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. Seamless integration with LangChain, the popular open-source framework for building with LLMs, is also provided. \n\nTo get started with testing in LangSmith, you can refer to the tracing and evaluation quick start guides provided in the platform's documentation. Other sections in the user guide also explain the workflows that LangSmith supports at each stage of the LLM application lifecycle.\n\nPlease note that before you begin, you'll need to create an account, obtain an API key, and configure your environment."}

In [19]:
agent_executor.invoke({"input": "Hvordan er været i Åros i Røyken, Norge?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current weather in Åros in Røyken, Norway'}`


[{'url': 'https://www.timeanddate.com/weather/norway/royken', 'content': 'Current weather in Røyken and forecast for today, tomorrow, and next 14 days'}, {'url': 'https://www.timeanddate.com/weather/norway/royken/ext', 'content': 'Røyken 14 Day Extended Forecast. Weather. Time Zone. DST Changes. Sun & Moon. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 42 °F. Cool. (Weather station: Hasle - MADIS Station, Norway).'}, {'url': 'https://weather.com/weather/tenday/l/Åros+Viken+Norway?canonicalCityId=89fab4fd4a4171e85b636661f200f9395bee4a1777531f2fe70171f4e2585a80', 'content': 'Be prepared with the most accurate 10-day forecast for Åros, Viken, Norway with highs, lows, chance of precipitation from The Weather Channel and Weather.com'}, {'url': 'https://www.wunderground.com/forecast/no/åros', 'conte

{'input': 'Hvordan er været i Åros i Røyken, Norge?',
 'output': 'Været i Åros i Røyken, Norge er for tiden kjølig med en temperatur på 42 °F (5.56 °C). For mer detaljert informasjon og en 10-dagers værprognose, kan du besøke [Weather Channel](https://weather.com/weather/tenday/l/Åros+Viken+Norway?canonicalCityId=89fab4fd4a4171e85b636661f200f9395bee4a1777531f2fe70171f4e2585a80) eller [Wunderground](https://www.wunderground.com/forecast/no/åros).'}

ADDING IN MEMORY

In [20]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [21]:
from langchain_core.messages import AIMessage, HumanMessage

In [22]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you further?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I assist you further?'}

KEEP TRACK OF MESSAGES AUTOMATICALLY

In [23]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [24]:
message_history = ChatMessageHistory()

In [25]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}